In [6]:
import pandas as pd, numpy as np
from copy import deepcopy
pred = pd.read_csv('/Users/yunfanz/Projects/army_challenge/results/test1_ens4.csv')


In [7]:
snr_pred = np.load('/Users/yunfanz/Projects/army_challenge/results/snr_preds.npy')
snr_pred = np.argmax(snr_pred, axis=1)
print(snr_pred.shape)

(100000,)


In [8]:
cols = pred.columns[1:]
pred.head(n=5)
print(cols, len(cols))


Index(['16PSK', '2FSK_5KHz', '2FSK_75KHz', '8PSK', 'AM_DSB', 'AM_SSB',
       'APSK16_c34', 'APSK32_c34', 'BPSK', 'CPFSK_5KHz', 'CPFSK_75KHz',
       'FM_NB', 'FM_WB', 'GFSK_5KHz', 'GFSK_75KHz', 'GMSK', 'MSK', 'NOISE',
       'OQPSK', 'PI4QPSK', 'QAM16', 'QAM32', 'QAM64', 'QPSK'],
      dtype='object') 24


In [ ]:
pred['PI4QPSK'] = 1.e-15
pred['FM_NB'] = 1.e-15  #set this next time, skipping to see effect
pred['FM_WB'] = 1.e-15

In [9]:
#confusion = ['QAM32','QAM64','QAM16','APSK16_c34','APSK32_c34']
confusion = ['8PSK','16PSK']
#confusion = ['CPFSK_5KHz', 'CPFSK_75KHz']
con_arr = np.sort([list(cols).index(col) for col in confusion])
con_arr

array([0, 3])

In [10]:
pred_arr = np.array(pred[cols])
print(pred_arr.shape)
pred_arr[1]

(100000, 24)


array([  1.85937449e-06,   1.00000000e-15,   9.62000000e-13,
         2.55135164e-05,   1.00000000e-15,   1.00000000e-15,
         1.58752918e-01,   4.40822452e-01,   1.00000000e-15,
         1.00000000e-15,   7.69200000e-12,   1.00000000e-15,
         3.80000000e-14,   8.04000000e-13,   4.98500000e-12,
         2.06810000e-11,   7.44000000e-13,   1.00000000e-15,
         9.31724000e-10,   3.13520000e-11,   5.95409721e-02,
         1.43808812e-01,   1.97047502e-01,   4.50500000e-12])

In [11]:
m = [[],[],[],[],[],[]]
for j, row in enumerate(pred_arr):
    if np.argmax(row) in con_arr:
        snr_cat = int(snr_pred[j])
        m[snr_cat].append(np.amax(row))
        
        group = deepcopy(row[con_arr])
        #print(group)
        for i in range(con_arr.size):
#            group[i] = 0.85*group[i] + 0.03*np.sum(row[con_arr]) 
            group[i] = 0.9*group[i] + 0.05*np.sum(row[con_arr]) 
        #print(group)
        
        #print(pred_arr[j][con_arr])
        pred_arr[j][con_arr] = group
        #print(pred_arr[j][con_arr])
        #break

In [12]:
def clip_prob(pred_a, eps):
    pred_a = np.where(pred_a<eps, eps, pred_a)
    pred_a = np.where(pred_a>1-eps, 1-eps, pred_a)
    pred_a /= np.sum(pred_a, axis=1, keepdims=True)
    return pred_a
pred_arr = clip_prob(pred_arr, eps=1.e-3)
pred_arr[:,11] = 1.e-15
pred_arr[:,12] = 1.e-15  #set this next time, skipping to see effect
pred_arr[:,19] = 1.e-15

In [42]:
for M in m:
    print(len(M), np.mean(M), np.std(M))

1190 0.127223415986 0.0276660268568
1746 0.315863076414 0.128237214223
1919 0.5664353915 0.165802574612
1871 0.700519364094 0.161521020647
1867 0.736168148767 0.166813592243
2022 0.747431489719 0.170546938513


In [13]:
new_pred = pd.DataFrame(pred_arr, columns=cols)
new_pred['Index'] = pred['Index']

In [14]:
new_pred.set_index('Index', inplace=True)
new_pred.head(n=5)

,16PSK,2FSK_5KHz,2FSK_75KHz,8PSK,AM_DSB,AM_SSB,APSK16_c34,APSK32_c34,BPSK,CPFSK_5KHz,...,GFSK_75KHz,GMSK,MSK,NOISE,OQPSK,PI4QPSK,QAM16,QAM32,QAM64,QPSK
Index,,,,,,,,,,,,,,,,,,,,,
1,0.000980,0.032022,0.000980,0.000980,0.000980,0.915033,0.000980,0.000980,0.000980,0.020924,...,0.000980,0.000980,0.000980,0.000980,0.000980,1.000000e-15,0.000980,0.000980,0.000980,0.000980
2,0.000981,0.000981,0.000981,0.000981,0.000981,0.000981,0.155797,0.432615,0.000981,0.000981,...,0.000981,0.000981,0.000981,0.000981,0.000981,1.000000e-15,0.058432,0.141131,0.193379,0.000981
3,0.000981,0.000981,0.000981,0.000981,0.000981,0.000981,0.160353,0.605787,0.000981,0.000981,...,0.000981,0.000981,0.000981,0.000981,0.000981,1.000000e-15,0.052788,0.108003,0.054423,0.000981
4,0.000978,0.000978,0.977495,0.000978,0.000978,0.000978,0.000978,0.000978,0.000978,0.000978,...,0.000978,0.000978,0.000978,0.000978,0.000978,1.000000e-15,0.000978,0.000978,0.000978,0.000978
5,0.000983,0.000983,0.000983,0.000983,0.405333,0.567598,0.000983,0.000983,0.000983,0.002006,...,0.000983,0.000983,0.000983,0.000983,0.000983,1.000000e-15,0.000983,0.000983,0.000983,0.000983


In [16]:
#new_pred.set_index('Index', inplace=True)
new_pred.to_csv('/Users/yunfanz/Projects/army_challenge/results/TestSet1Predictions.csv',float_format='%.8f')

In [86]:
pred_mods = np.argmax(pred_arr, axis=1)

In [30]:
np.histogram(snr_pred)

(array([16726,     0, 16774,     0, 16720,     0, 16969,     0, 16228, 16583]),
 array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ]))

In [96]:
np.array([p in con_arr for p in pred_mods]).nonzero()[0][0]

1

In [91]:
con_arr

array([ 6,  7, 20, 21, 22])